In [3]:
"""
README: YOLO Worst-10 Detection Visualizer by City and Class

This script:
- For each class, evaluates detection performance by comparing YOLO-format predictions with ground truth per image,
- Calculates the match ratio (detected GT boxes / total GT boxes) for each class and city,
- Saves the **worst 10** (lowest match ratio) visualizations per class/city as:
    - original.jpg
    - labels.jpg (ground-truth boxes overlayed in green)
    - detections.jpg (detections overlayed in class color)
- Output is organized by [class]/[city]/[ranking] in the output folder.

How to use:
- Set `labels_dir`, `gt_dir`, `images_dir`, and `output_dir` as appropriate.
- Adjust city keywords and class definitions as needed.
- Run the script; results will be saved under OUTPUT_DIR.

Requirements:
- Python 3.7+
- OpenCV
- tqdm
- pathlib

Author: Bahadir Akin Akgul
Date: 13.07.2025
"""

import cv2
import time
from pathlib import Path
from datetime import datetime
from collections import defaultdict

# === SETTINGS ===
labels_dir = Path('PATH_TO_PREDICTION_LABELS')
gt_dir = Path('PATH_TO_GROUND_TRUTH_LABELS')
images_dir = Path('PATH_TO_TEST_IMAGES')
output_dir = Path('yolo-worst-10-by-city-class')
output_dir.mkdir(parents=True, exist_ok=True)

iou_threshold = 0.5
class_names = ['pedestrian', 'road', 'vehicle']
class_ids = {'pedestrian': 0, 'road': 1, 'vehicle': 2}
colors = {
    'gt': (0, 255, 0),       # ground truth - green
    0: (0, 255, 255),        # pedestrian - yellow
    1: (255, 0, 255),        # road - magenta
    2: (0, 0, 255)           # vehicle - red
}

city_keywords = {
    'istanbul': ['libadiye', 'levent', 'taksim', 'ciragan', 'barbaros', 'dolmabahce', 'bagdat', 'muallim', 'katar'],
    'paris': ['paris-champs'],
    'munich': ['munih'],
    'marseille': ['marsilya']
}
city_translation = {
    'istanbul': 'Istanbul',
    'paris': 'Paris',
    'munich': 'Munich',
    'marseille': 'Marseille',
    'unknown': 'Unknown'
}

def now():
    return datetime.now().strftime('%H:%M:%S')

def calculate_iou(boxA, boxB):
    xA, yA, wA, hA = boxA
    xa1, ya1, xa2, ya2 = xA - wA / 2, yA - hA / 2, xA + wA / 2, yA + hA / 2
    xB, yB, wB, hB = boxB
    xb1, yb1, xb2, yb2 = xB - wB / 2, yB - hB / 2, xB + wB / 2, yB + hB / 2

    inter_x1 = max(xa1, xb1)
    inter_y1 = max(ya1, yb1)
    inter_x2 = min(xa2, xb2)
    inter_y2 = min(ya2, yb2)
    inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)
    area_a = (xa2 - xa1) * (ya2 - ya1)
    area_b = (xb2 - xb1) * (yb2 - yb1)
    return inter_area / (area_a + area_b - inter_area + 1e-6)

def get_city(name):
    for city, keywords in city_keywords.items():
        if any(k in name for k in keywords):
            return city
    return 'unknown'

def draw_boxes(img, boxes, classes, cls_id, color):
    h, w = img.shape[:2]
    for c, b in zip(classes, boxes):
        if c != cls_id:
            continue
        x, y, bw, bh = b
        x1 = int((x - bw/2) * w)
        y1 = int((y - bh/2) * h)
        x2 = int((x + bw/2) * w)
        y2 = int((y + bh/2) * h)
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 1)
    return img

print(f"[{now()}] Starting worst-10 detection visualization...")

for cls_name in class_names:
    cls_id = class_ids[cls_name]
    results_by_city = defaultdict(list)

    for txt_file in labels_dir.glob("*.txt"):
        name = txt_file.stem
        city = get_city(name)
        gt_file = gt_dir / txt_file.name
        image_file = next(images_dir.glob(f"{name}.*"), None)

        if not gt_file.exists() or not image_file:
            continue

        with open(gt_file) as f:
            gt_lines = [line.strip().split() for line in f if line.strip()]
        gt_classes = [int(l[0]) for l in gt_lines]
        gt_boxes = [list(map(float, l[1:5])) for l in gt_lines]

        with open(txt_file) as f:
            det_lines = [line.strip().split() for line in f if line.strip()]
            det_classes = []
            det_boxes = []
            for l in det_lines:
                cls = int(l[0])
                conf = float(l[5]) if len(l) > 5 else 1.0  # Default conf = 1.0 if not available
                if conf > 0.5:
                    det_classes.append(cls)
                    det_boxes.append(list(map(float, l[1:5])))

        matched = 0
        total_gt = sum(1 for c in gt_classes if c == cls_id)
        for i, gt_box in enumerate(gt_boxes):
            if gt_classes[i] != cls_id:
                continue
            for j, det_box in enumerate(det_boxes):
                if det_classes[j] != cls_id:
                    continue
                if calculate_iou(gt_box, det_box) >= iou_threshold:
                    matched += 1
                    break

        match_ratio = matched / total_gt if total_gt else 1.0

        if total_gt > 0:
            img = cv2.imread(str(image_file))
            gt_img = draw_boxes(img.copy(), gt_boxes, gt_classes, cls_id, colors['gt'])
            det_img = draw_boxes(img.copy(), det_boxes, det_classes, cls_id, colors[cls_id])

            results_by_city[city].append({
                "name": name,
                "match_ratio": match_ratio,
                "original": img,
                "gt": gt_img,
                "det": det_img
            })

    for city, items in results_by_city.items():
        city_label = city_translation.get(city, city)
        city_output = output_dir / cls_name / city_label
        city_output.mkdir(parents=True, exist_ok=True)

        worst10 = sorted(items, key=lambda x: x['match_ratio'])[:10]
        for i, item in enumerate(worst10, 1):
            subdir = city_output / f"{i:02d}_{item['name']}"
            subdir.mkdir(parents=True, exist_ok=True)
            cv2.imwrite(str(subdir / "original.jpg"), item["original"])
            cv2.imwrite(str(subdir / "labels.jpg"), item["gt"])
            cv2.imwrite(str(subdir / "detections.jpg"), item["det"])

        print(f"[{now()}] {cls_name.upper()} - {city_label}: {len(worst10)} results saved")

print(f"[{now()}] Visualization complete for all classes.")


[23:55:35] 🔍 Başlıyor...
[23:55:51] ✅ PEDESTRIAN - Istanbul: 10 kayıt
[23:55:51] ✅ PEDESTRIAN - Munich: 10 kayıt
[23:55:52] ✅ PEDESTRIAN - Paris: 10 kayıt
[23:55:52] ✅ PEDESTRIAN - Marseille: 10 kayıt
[23:55:52] ✅ PEDESTRIAN - Unknown: 1 kayıt
[23:56:19] ✅ ROAD - Istanbul: 10 kayıt
[23:56:19] ✅ ROAD - Munich: 10 kayıt
[23:56:20] ✅ ROAD - Paris: 10 kayıt
[23:56:20] ✅ ROAD - Marseille: 10 kayıt
[23:56:20] ✅ ROAD - Unknown: 1 kayıt
[23:56:43] ✅ VEHICLE - Istanbul: 10 kayıt
[23:56:43] ✅ VEHICLE - Munich: 10 kayıt
[23:56:44] ✅ VEHICLE - Paris: 10 kayıt
[23:56:44] ✅ VEHICLE - Marseille: 10 kayıt
[23:56:44] ✅ VEHICLE - Unknown: 1 kayıt
[23:56:44] ✅ Tüm sınıflar için işlem tamamlandı.


In [1]:
import cv2
import json
import time
from pathlib import Path
from datetime import datetime
from collections import defaultdict
from PIL import Image

# === AYARLAR ===
labels_dir = Path('runs/detect/yolov10-txtresults/labels')
gt_dir = Path('/truba/home/baakgul/roadtr-14032025/test/labels')
images_dir = Path('/truba/home/baakgul/roadtr-14032025/test/images')
output_dir = Path('yolo-worst-10-by-city-class-300dpi')  # 300DPI olarak güncellendi
output_dir.mkdir(parents=True, exist_ok=True)

iou_threshold = 0.5
class_names = ['pedestrian', 'road', 'vehicle']
class_ids = {'pedestrian': 0, 'road': 1, 'vehicle': 2}
colors = {
    'gt': (0, 255, 0),       # ground truth - green
    0: (0, 255, 255),        # pedestrian - yellow
    1: (255, 0, 255),        # road - pink (instead of black)
    2: (0, 0, 255)           # vehicle - red
}

city_keywords = {
    'istanbul': ['libadiye', 'levent', 'taksim', 'ciragan', 'barbaros', 'dolmabahce', 'bagdat', 'muallim', 'katar'],
    'paris': ['paris-champs'],
    'munih': ['munih'],
    'marsilya': ['marsilya']
}
city_translation = {
    'istanbul': 'Istanbul',
    'paris': 'Paris',
    'munih': 'Munich',
    'marsilya': 'Marseille',
    'unknown': 'Unknown'
}

def now():
    return datetime.now().strftime('%H:%M:%S')

def calculate_iou(boxA, boxB):
    xA, yA, wA, hA = boxA
    xa1, ya1, xa2, ya2 = xA - wA / 2, yA - hA / 2, xA + wA / 2, yA + hA / 2
    xB, yB, wB, hB = boxB
    xb1, yb1, xb2, yb2 = xB - wB / 2, yB - hB / 2, xB + wB / 2, yB + hB / 2

    inter_x1 = max(xa1, xb1)
    inter_y1 = max(ya1, yb1)
    inter_x2 = min(xa2, xb2)
    inter_y2 = min(ya2, yb2)
    inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)
    area_a = (xa2 - xa1) * (ya2 - ya1)
    area_b = (xb2 - xb1) * (yb2 - yb1)
    return inter_area / (area_a + area_b - inter_area + 1e-6)

def get_city(name):
    for city, keywords in city_keywords.items():
        if any(k in name for k in keywords):
            return city
    return 'unknown'

def draw_boxes(img, boxes, classes, cls_id, color):
    h, w = img.shape[:2]
    for c, b in zip(classes, boxes):
        if c != cls_id:
            continue
        x, y, bw, bh = b
        x1 = int((x - bw/2) * w)
        y1 = int((y - bh/2) * h)
        x2 = int((x + bw/2) * w)
        y2 = int((y + bh/2) * h)
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 1)
    return img

def save_with_dpi(img_array, path, dpi=(300, 300)):
    img_rgb = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
    img_pil = Image.fromarray(img_rgb)
    img_pil.save(path, dpi=dpi, quality=95)

print(f"[{now()}] 🔍 Başlıyor...")

for cls_name in class_names:
    cls_id = class_ids[cls_name]
    results_by_city = defaultdict(list)

    for txt_file in labels_dir.glob("*.txt"):
        name = txt_file.stem
        city = get_city(name)
        gt_file = gt_dir / txt_file.name
        image_file = next(images_dir.glob(f"{name}.*"), None)

        if not gt_file.exists() or not image_file:
            continue

        with open(gt_file) as f:
            gt_lines = [line.strip().split() for line in f if line.strip()]
        gt_classes = [int(l[0]) for l in gt_lines]
        gt_boxes = [list(map(float, l[1:5])) for l in gt_lines]

        with open(txt_file) as f:
            det_lines = [line.strip().split() for line in f if line.strip()]
            det_classes = []
            det_boxes = []
            for l in det_lines:
                cls = int(l[0])
                conf = float(l[5]) if len(l) > 5 else 1.0
                if conf > 0.5:
                    det_classes.append(cls)
                    det_boxes.append(list(map(float, l[1:5])))

        matched = 0
        total_gt = sum(1 for c in gt_classes if c == cls_id)
        for i, gt_box in enumerate(gt_boxes):
            if gt_classes[i] != cls_id:
                continue
            for j, det_box in enumerate(det_boxes):
                if det_classes[j] != cls_id:
                    continue
                if calculate_iou(gt_box, det_box) >= iou_threshold:
                    matched += 1
                    break

        match_ratio = matched / total_gt if total_gt else 1.0

        if total_gt > 0:
            img = cv2.imread(str(image_file))
            gt_img = draw_boxes(img.copy(), gt_boxes, gt_classes, cls_id, colors['gt'])
            det_img = draw_boxes(img.copy(), det_boxes, det_classes, cls_id, colors[cls_id])

            results_by_city[city].append({
                "name": name,
                "match_ratio": match_ratio,
                "original": img,
                "gt": gt_img,
                "det": det_img
            })

    for city, items in results_by_city.items():
        city_label = city_translation.get(city, city)
        city_output = output_dir / cls_name / city_label
        city_output.mkdir(parents=True, exist_ok=True)

        worst10 = sorted(items, key=lambda x: x['match_ratio'])[:10]
        for i, item in enumerate(worst10, 1):
            subdir = city_output / f"{i:02d}_{item['name']}"
            subdir.mkdir(parents=True, exist_ok=True)

            save_with_dpi(item["original"], str(subdir / "original.jpg"))
            save_with_dpi(item["gt"], str(subdir / "labels.jpg"))
            save_with_dpi(item["det"], str(subdir / "detections.jpg"))

        print(f"[{now()}] ✅ {cls_name.upper()} - {city_label}: {len(worst10)} kayıt")

print(f"[{now()}] ✅ Tüm sınıflar için işlem tamamlandı.")


[01:11:39] 🔍 Başlıyor...
[01:12:50] ✅ PEDESTRIAN - Istanbul: 10 kayıt
[01:12:50] ✅ PEDESTRIAN - Munich: 10 kayıt
[01:12:51] ✅ PEDESTRIAN - Paris: 10 kayıt
[01:12:51] ✅ PEDESTRIAN - Marseille: 10 kayıt
[01:12:51] ✅ PEDESTRIAN - Unknown: 1 kayıt
[01:13:27] ✅ ROAD - Istanbul: 10 kayıt
[01:13:28] ✅ ROAD - Munich: 10 kayıt
[01:13:28] ✅ ROAD - Paris: 10 kayıt
[01:13:29] ✅ ROAD - Marseille: 10 kayıt
[01:13:29] ✅ ROAD - Unknown: 1 kayıt
[01:13:50] ✅ VEHICLE - Istanbul: 10 kayıt
[01:13:51] ✅ VEHICLE - Munich: 10 kayıt
[01:13:51] ✅ VEHICLE - Paris: 10 kayıt
[01:13:52] ✅ VEHICLE - Marseille: 10 kayıt
[01:13:52] ✅ VEHICLE - Unknown: 1 kayıt
[01:13:52] ✅ Tüm sınıflar için işlem tamamlandı.
